<a href="https://colab.research.google.com/github/rlaalswn222/kaggle-dacon-practice/blob/main/dacon_NovelistClassification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install matplotlib-venn

In [14]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [15]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [16]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [17]:
!pip install cartopy
import cartopy

In [18]:
!pip install keras

In [19]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings(action='ignore')

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.NLP Preprocessing

In [21]:
import pandas as pd

path = '/content/drive/MyDrive/df/dacon_NovelistClassification/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test_x.csv')
sample_submission = pd.read_csv(path + "sample_submission.csv")

In [27]:
#파일 불러오기
raw_len=len(train)
train['length'] = train['text'].apply(lambda x:len(x.split()))

test['length'] = test['text'].apply(lambda x:len(x.split()))

In [23]:
!pip install tqdm

In [29]:
from tqdm import tqdm

MAX_LENGTH = 200
text_split=[]
author_split=[]
index=[]
cnt=0
for i, (text_tmp, length_tmp, author_tmp) in tqdm(enumerate(zip(train['text'].values, train['length'].values, train['author'].values))):
    if length_tmp>=MAX_LENGTH:                                 # 설정한 MAX_LENGTH가 넘어가면
        n_split = (length_tmp-1)//MAX_LENGTH+1                 # 문장을 분리한다.
        idx = length_tmp//2                                    # 분리된 인덱스
        r=int(length_tmp*0.05)
        for n in range(n_split):
            text_split.append(text_tmp[idx*n-r:idx*(n+1)+r])   # 분리하여 리스트에 저장
            author_split.append(author_tmp)
        cnt+=1
    else:
        index.append(i)                                          # MAX_LENGTH 넘어가지 않으면 그 인덱스를 저장해둔다.

54879it [00:00, 131873.70it/s]


In [30]:
# 분리한 문장으로 train dataframe을 재정의
new_text = np.array((list(train.iloc[index]['text']) + text_split))
new_author = np.array((list(train.iloc[index]['author']) + author_split))
train = pd.DataFrame({'text':new_text, 'author':new_author})

In [31]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)

In [32]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as",
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could",
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has",
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself",
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself",
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours",
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that",
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll",
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll",
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [33]:
#전처리 적용
train['text'] = train['text'].str.lower()
train['text'] = train['text'].apply(alpha_num)
test['text'] = test['text'].str.lower()
test['text'] = test['text'].apply(alpha_num)

In [34]:
# train test 분리
X = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y = np.array([x for x in train['author']])

In [35]:
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Input,Conv1D,Embedding, GlobalAveragePooling1D, BatchNormalization, Bidirectional,LSTM,GRU
from tensorflow.keras.models import Model
import re
from sklearn.model_selection import train_test_split, StratifiedKFold
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras.callbacks import ModelCheckpoint
import seaborn as sns


In [36]:
# Parameters
vocab_size = 20000
embedding_dim = 64
max_length = MAX_LENGTH

In [37]:
# Tokenizing
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index

In [38]:
# text to sequence
# padding
train_sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(train_sequences, padding='post', maxlen=max_length)


test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [39]:
# define model
def build_model():
    INPUT = Input(shape=X.shape[1:])
    emb = Embedding(vocab_size, embedding_dim, input_length=max_length)(INPUT)
    conv = Conv1D(32, 2, activation='relu', padding='same')(emb)
    pool = GlobalAveragePooling1D()(emb)
    drop = tf.keras.layers.Dropout(0.2)(pool)
    dense = Dense(32, activation='relu')(drop)
    OUTPUT = Dense(5, activation='softmax')(dense)
    model = Model(INPUT, OUTPUT)
    return model

In [40]:
# split data
skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(X,y):
    folds.append((train_idx, valid_idx))

In [41]:
fold=0
train_idx, valid_idx = folds[fold][0], folds[fold][1]
X_train, X_valid, y_train, y_valid = X[train_idx], X[valid_idx], y[train_idx], y[valid_idx]

batch_size=256
epochs=40

decay_st = (len(X_train)//batch_size+1)*epochs
poly_sche = tf.keras.optimizers.schedules.PolynomialDecay(1e-3, decay_st, end_learning_rate=1e-6, power=0.9)
opt = tf.keras.optimizers.Adam(poly_sche)

In [42]:
model = build_model()
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 200, 64)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,282,245 (4.89 MB)

 Trainable params: 1,282,245 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

None


In [43]:
# validation loss 를 계속 비교하면서 개선될 때 마다 저장
model_name = 'conv_model'
cp_name = 'model/{}-{}'.format(model_name, fold)
# Change the file extension to '.keras'
cp = ModelCheckpoint(cp_name + ".keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [44]:
# 훈련이 되는지 test run
# fit model
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    validation_data=([X_valid, y_valid]), batch_size=batch_size, callbacks=[cp])

Epoch 1/40
175/176 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2691 - loss: 1.5698
Epoch 1: val_loss improved from inf to 1.54141, saving model to model/conv_model-0.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.2692 - loss: 1.5697 - val_accuracy: 0.2888 - val_loss: 1.5414
Epoch 2/40
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3185 - loss: 1.5206
Epoch 2: val_loss improved from 1.54141 to 1.43415, saving model to model/conv_model-0.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.3186 - loss: 1.5205 - val_accuracy: 0.3795 - val_loss: 1.4341
Epoch 3/40
175/176 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.3953 - loss: 1.4117
Epoch 3: val_loss improved from 1.43415 to 1.30904, saving model to model/conv_model-0.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.3955 - loss: 1.4114 - val_accuracy: 0.4959 - val_loss: 1.3090
Epoch 4/40
174/176 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4858 - loss: 1.2786
Epoch 4: val_loss impro

In [ ]:
# 5-fold 학습
for fold in range(5):
    train_idx, valid_idx = folds[fold][0], folds[fold][1]
    X_train, X_valid, y_train, y_valid = X[train_idx], X[valid_idx], y[train_idx], y[valid_idx]

    batch_size=256
    epochs=40

    decay_st = (len(X_train)//batch_size+1)*epochs
    poly_sche = tf.keras.optimizers.schedules.PolynomialDecay(1e-3, decay_st, end_learning_rate=1e-6, power=0.9)
    opt = tf.keras.optimizers.Adam(poly_sche)

    model = build_model()

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    print(model.summary())

    model_name = 'conv_model'
    cp_name = 'model/{}-{}'.format(model_name, fold)
    cp = ModelCheckpoint(cp_name + ".keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # fit model
    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        validation_data=([X_valid, y_valid]), batch_size=batch_size, callbacks=[cp])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 200, 64)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,282,245 (4.89 MB)

 Trainable params: 1,282,245 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, 200, 64)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_5           │ (None, 64)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,282,245 (4.89 MB)

 Trainable params: 1,282,245 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/40
175/176 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2694 - loss: 1.5712
Epoch 1: val_loss improved from inf to 1.54117, saving model to model/conv_model-4.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.2696 - loss: 1.5711 - val_accuracy: 0.3019 - val_loss: 1.5412
Epoch 2/40
175/176 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3224 - loss: 1.5243
Epoch 2: val_loss improved from 1.54117 to 1.43218, saving model to model/conv_model-4.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.3226 - loss: 1.5240 - val_accuracy: 0.3896 - val_loss: 1.4322
Epoch 3/40
174/176 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4044 - loss: 1.4088
Epoch 3: val_loss improved from 1.43218 to 1.30446, saving model to model/conv_model-4.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.4047 - loss: 1.4083 - val_accuracy: 0.4488 - val_loss: 1.3045
Epoch 4/40
174/176 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4758 - loss: 1.2772
Epoch 4: val_loss

In [46]:
import os

# Create the 'model' directory if it doesn't exist
if not os.path.exists('model'):
    os.makedirs('model')

test_pred=[]
for fold in range(5):
    # Change the file extension to '.keras'
    model.load_weights('model/conv_model-{}.keras'.format(fold))
    train_idx, valid_idx = folds[fold][0], folds[fold][1]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    model.evaluate(X_valid, y_valid)
    test_pred.append(model.predict(test_padded))

352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7491 - loss: 0.6790
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7452 - loss: 0.6949
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7561 - loss: 0.6702
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7582 - loss: 0.6552
614/614 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7483 - loss: 0.6869
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [47]:
test_pred=np.array(test_pred)

In [55]:
# 지정한 폴더에 CSV 파일로 저장 (파일이 없으면 새로 생성됩니다)
file_path = '/content/drive/MyDrive/df/dacon_NovelistClassification/sample_submission111.csv'
submit.to_csv(file_path, index=False)

print(f"파일이 생성되었습니다: {file_path}")

파일이 생성되었습니다: /content/drive/MyDrive/df/dacon_NovelistClassification/sample_submission111.csv


In [56]:
submit=pd.read_csv('/content/drive/MyDrive/df/dacon_NovelistClassification/sample_submission111.csv')
submit.iloc[:,1:]=test_pred.mean(0)
submit.to_csv('submission.csv', index=False)

In [57]:
 #모델 평가 (정확도 출력)
loss, accuracy = model.evaluate(X_valid, y_valid)
print(f"✅ 테스트 데이터 정확도: {accuracy:.4f}")

352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7483 - loss: 0.6869
✅ 테스트 데이터 정확도: 0.7372
